**本次比赛整体方案参考AIStuidio用户“红白黑”的项目“常规赛：PALM病理性近视预测基线方案”。<br>项目地址：
[常规赛：PALM病理性近视预测基线](https://aistudio.baidu.com/aistudio/projectdetail/1968869?channelType=0&channel=0)**

# 0.赛题简介：
```
PALM病理性近视预测常规赛的重点是研究和发展与病理性近视诊断相关的算法。该常规赛的目标是评估和比较在一个常见的视网膜眼底图像数据集上检测病理性近视的自动算法。具体任务是将提供的图像分为病理性近视眼底彩照和非病理性近视眼底彩照，其中，非病理性近视眼底彩照包括正常眼底和高度近视眼底彩照。
```
![](https://ai-studio-static-online.cdn.bcebos.com/debda405c1d149d5974909f6e5f69d15643c9cab5fc74c4fbbf29c6115e839a0)

---
题干提到，该比赛的目标是“将提供的图像分为病理性、非病理性彩照”，其中非病理性数据有两类，均标记为“非病理性”。本质上来说，可以将赛题理解为一个二分类任务。

# 1.导入开发套件PaddleCLas
* 若无法从GitHub上顺利导入套件，可从码云导入。

In [4]:
# !git clone https://github.com/PaddlePaddle/PaddleClas.git
!git clone https://gitee.com/paddlepaddle/PaddleClas.git -b develop

* 解压数据集，并将其放入PaddleClas的dataset文件夹中

In [7]:
!cd /home/aistudio/

!unzip -oq /home/aistudio/data/data93287/常规赛：PALM病理性近视预测.zip
!mv 常规赛：PALM病理性近视预测 PaddleClas/dataset/
!rm -rf PaddleClas/dataset/__MACOSX

rm: cannot remove '__MACOSX': Is a directory


# 2.数据集处理

* PALM病理性近视预测常规赛由中山大学中山眼科中心提供800张带病理性近视分类标注的眼底彩照供选手训练模型，另提供400张带标注数据供平台进行模型测试
* Train文件夹中包含fundus_image文件夹和Classification.xlsx文件。其中fundus_image文件夹存放训练集，Classification.xlsx文件中为各眼底图像是否属于病理性近视，属于为1，不属于为0
* PALM-Testing400-Images 文件夹包含测试集文件，共400张眼底彩照，命名形如T0001.jpg。

* 从训练集中划分出一部分数据作为验证集，len(训练集：验证集) = 8：2

In [8]:
import pandas as pd
import os
from tqdm import tqdm

random_seed = 1024

Image_path = 'Train/fundus_image'                             # 数据集存放路径
total_data = pd.read_excel('PaddleClas/dataset/常规赛：PALM病理性近视预测/Train/Classification.xlsx')       # 读取数据集label文件，xlsx格式

for i in range(len(total_data)):                                                         # 将DataFrame表格中的图片补足路径
    total_data.iloc[i, 0] = os.path.join(Image_path, total_data.iloc[i, 0])              # 拼接路径

total_data = total_data.sample(frac=1.0, random_state=random_seed).reset_index(drop=True)       # frac=1.0对应随机采样全部样本（表格数据），对应打乱表格


# 训练集：验证集占比设定
train_percent = 0.8

train_data = total_data.to_numpy()[:int(len(total_data) * train_percent)] # 要保存的训练数据
eval_data = total_data.to_numpy()[int(len(total_data) * train_percent):]  # 要保存的验证数据

train_txt_save_path = 'PaddleClas/dataset/常规赛：PALM病理性近视预测/train_list.txt' # 训练集label文件存放路径
val_txt_save_path = 'PaddleClas/dataset/常规赛：PALM病理性近视预测/val_list.txt' # 验证集label文件存放路径

with open(train_txt_save_path, 'w') as f:
    t_tq = tqdm(train_data)
    t_tq.set_description('Train Data Spliting')
    for i in t_tq:
        f.write(i[0]+' '+str(i[1])+'\n')

with open(val_txt_save_path, 'w') as f:
    v_tq = tqdm(eval_data)
    v_tq.set_description('Val Data Spliting')
    for i in v_tq:
        f.write(i[0]+' '+str(i[1])+'\n')

Val Data Spliting: 100%|██████████| 160/160 [00:00<00:00, 137602.76it/s]


In [9]:
# 查看数据集大小

print('length of training data: ', len(train_data))
print('length of eval_data: ', len(eval_data))

length of training data:  640
length of eval_data:  160


# 4.模型选择
模型选用：DeiTDeiT_base_distilled_patch16_384。<br>
其配置文件路径：PaddleClas/configs/DeiT/DeiT_base_distilled_patch16_384.yaml

In [ ]:
# 以下为更改后的yaml文件内容
# 若需修改配置，将其更改后替换原yaml文件内容

'''
mode: 'train'
ARCHITECTURE:
    name: 'DeiT_base_distilled_patch16_384'

# 预训练模型：本项目默认不使用预训练模型
pretrained_model: ""
model_save_dir: "./output/"  # 模型训练参数结果存放路径

classes_num: 2       # 修改分类数，本次任务为2分类

total_images:800        # 修改数据集大小 
save_interval: 1        # 模型参数保存频率：
validate: True          # 是否开启评估
valid_interval: 1       # 评估频率
epochs: 120             # 总训练轮数
topk: 2                 # 
image_shape: [3, 384, 384]

use_mix: False
ls_epsilon: -1

# 学习率设定
LEARNING_RATE:
    function: 'Cosine'          
    params:                   
        lr: 0.01               

# 优化器设定
OPTIMIZER:
    function: 'Momentum'
    params:
        momentum: 0.9
    regularizer:
        function: 'L2'   # L2正则化
        factor: 0.000100


# 浅谈num_workers参数：
# dataloader一次性创建num_worker个worker;
# 也可以说dataloader一次性创建num_worker个工作进程，worker也是普通的工作进程，
# 并用batch_sampler将指定batch分配给指定worker，worker将它负责的batch加载进RAM。

# num_worker设置得大，好处是寻batch速度快，因为下一轮迭代的batch很可能在上一轮/上上一轮...迭代时已经加载好了；
# 坏处是内存开销大，也加重了CPU负担。

# 如果num_worker设为0，意味着每一轮迭代时，dataloader不再有自主加载数据到RAM这一步骤（因为没有worker了），
# 而是在RAM中找batch，找不到时再加载相应的batch。缺点当然是速度更慢。
num_workers的设定，取决于运行环境的硬件配置是否吃得消，尤其是内存；这里参数为PaddleClas里默认给出的4。


# 训练策略配置
TRAIN:
    batch_size: 64                                                 # 数据读取批次大小
    num_workers: 4
    file_list: "./dataset/常规赛：PALM病理性近视预测/train_list.txt" # 训练数据label文件路径
    data_dir: "./dataset/常规赛：PALM病理性近视预测/"                # 训练集数据路径
    shuffle_seed: 0
    transforms:                                                    # transform设定：数据增强方案
        - DecodeImage:             # 图像解码
            to_rgb: True
            channel_first: False
        - RandCropImage:           # 随机裁剪
            size: 384
        - RandFlipImage:           # 随机翻转
            flip_code: 1
        - NormalizeImage:          # 图像数值标准化
            scale: 1./255.
            mean: [0.485, 0.456, 0.406]
            std: [0.229, 0.224, 0.225]
            order: ''
        - ToCHWImage:

# 验证策略设定
VALID:
    batch_size: 64
    num_workers: 4
    file_list: "./dataset/常规赛：PALM病理性近视预测/val_list.txt" # 验证数据label文件路径
    data_dir: "./dataset/常规赛：PALM病理性近视预测/"              # 验证集数据路径
    shuffle_seed: 0
    transforms:
        - DecodeImage:
            to_rgb: True
            channel_first: False
        - ResizeImage:            # 图像尺寸变形
            resize_short: 426
        - CropImage:              # 图像裁剪
            size: 384
        - NormalizeImage:
            scale: 1.0/255.0
            mean: [0.485, 0.456, 0.406]
            std: [0.229, 0.224, 0.225]
            order: ''
        - ToCHWImage:

'''

# 5.模型训练
* 使用tools/train.py进行训练
* 参数列表：<br>
-c 'yaml配置文件路径'<br>
-o '预训练模型路径，不使用则不填'<br>
-o '是否使用GPU'

In [5]:
%cd /home/aistudio/PaddleClas

!python tools/train.py \
    -c configs/DeiT/DeiT_base_distilled_patch16_384.yaml \
    -o use_gpu=True 

# 6.模型预测
PaddleClas在tools中给出infer.py文件进行预测，更改后可直接生成csv文件用于提交。

In [ ]:
# 预测文件infer_V2.py内容

'''
import numpy as np
import cv2
import os
import sys

import paddle
import paddle.nn.functional as F

__dir__ = os.path.dirname(os.path.abspath(__file__))
sys.path.append(__dir__)
sys.path.append(os.path.abspath(os.path.join(__dir__, '../..')))

from ppcls.utils.save_load import load_dygraph_pretrain
from ppcls.utils import logger
from ppcls.modeling import architectures
from utils import parse_args, get_image_list, preprocess, postprocess, save_prelabel_results


def main():
    args = parse_args()
    # assign the place
    place = paddle.set_device('gpu' if args.use_gpu else 'cpu')
    multilabel = True if args.multilabel else False

    net = architectures.__dict__[args.model](class_dim=args.class_num)
    load_dygraph_pretrain(net, args.pretrained_model, args.load_static_weights)
    image_list = get_image_list(args.image_file)
    batch_input_list = []
    img_path_list = []
    cnt = 0

    # 添加结果列表，保存预测结果**********************
    submit_list = []
    # *********************************************
    id_info = 0  # 计数
    for idx, img_path in enumerate(image_list):
        img = cv2.imread(img_path)
        if img is None:
            logger.warning(
                "Image file failed to read and has been skipped. The path: {}".
                format(img_path))
            continue
        else:
            img = img[:, :, ::-1]
            data = preprocess(img, args)
            batch_input_list.append(data)
            img_path_list.append(img_path)
            cnt += 1

        if cnt % args.batch_size == 0 or (idx + 1) == len(image_list):
            batch_tensor = paddle.to_tensor(batch_input_list)
            net.eval()
            batch_outputs = net(batch_tensor)
            if args.model == "GoogLeNet":
                batch_outputs = batch_outputs[0]
            if multilabel:
                batch_outputs = F.sigmoid(batch_outputs)
            else:
                batch_outputs = F.softmax(batch_outputs)
            batch_outputs = batch_outputs.numpy()
            batch_result_list = postprocess(batch_outputs, args.top_k, multilabel=multilabel)

            for number, result_dict in enumerate(batch_result_list):
                filename = img_path_list[number].split("/")[-1]
                clas_ids = result_dict["clas_ids"]
                id_info += 1
                if multilabel:
                    print("File:{}, multilabel result: ".format(filename))
                    for id, score in zip(clas_ids, result_dict["scores"]):
                        print("\tclass id: {}, probability: {:.2f}".format(id, score))
                else:
                    scores_str = "[{}]".format(", ".join("{:.2f}".format(
                        r) for r in result_dict["scores"]))
                    print("Id:{}, File:{}, Top-{} result: class id(s): {}, score(s): {}".
                        format(id_info, filename, args.top_k, clas_ids, scores_str))

                    # 保存提交文件 -- 保存病理的结果********************
                    submit_list.append([filename, float(clas_ids[0])])  # 加入当前预测结果
                    # *********************************************
                    print(filename, float(clas_ids[0]))

                if args.pre_label_image:
                    save_prelabel_results(clas_ids[0], img_path_list[number],
                                          args.pre_label_out_idr)

            batch_input_list = []
            img_path_list = []
    # 转换数据类型 *********************************
    submit_list = np.asarray(submit_list)

    import pandas as pd
    Submit_data = pd.DataFrame(submit_list)          # 转为DataFrame格式
    Submit_data.columns = ['FileName', 'PM Risk']    # 修改列名
    Submit_data = Submit_data.sort_values(by='FileName').reset_index(drop=True)   # 按照图片id排列
    Submit_data.to_csv('Classification_Results.csv', index=False, float_format="%.1f")       # 保存结果csv
    # *********************************************


if __name__ == "__main__":
    main()
'''

In [18]:
# 将文件移入infer文件夹中

!cp infer_V2.py PaddleClas/tools/infer/

参数说明：<br>
* --model：模型名称，此处选用DeiT_base_distilled_patch16_384
* --i：训练集路径
* --pretrained_model：模型权重文件路径。使用之前训练好的模型进行预测。
* --use_gpu：是否使用GPU
* --class_num：分类数目。本次任务为二分类任务，class_num - 2
* --resize_short：对输入图像进行等比例缩放，表示最短边的尺寸；默认值256
* --resize: 对resize_short操作后的进行居中裁剪，表示裁剪的尺寸；默认值224
* --pre_label_image : 是否对图像数据进行预标注，默认值False
* pre_label_out_idr : 预标注图像数据的输出文件夹，当pre_label_image=True时，会在该文件夹下面生成很多个子文件夹，每个文件夹名称为类别id，其中存储模型预测**属于该类别的所有图像。**（适用于多分类任务的数据划分）

In [1]:
# !cd /home/aistudio/PaddleClas/

!python PaddleClas/tools/infer/infer_V2.py \
    --model DeiT_base_distilled_patch16_384 \
    -i PaddleClas/dataset/常规赛：PALM病理性近视预测/PALM-Testing400-Images \
    --pretrained_model PaddleClas/output/DeiT_base_distilled_patch16_384/best_model/ppcls  \
    --use_gpu True\
    --class_num 2 \
    --resize_short=426 \
    --resize=384

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
W0704 22:53:11.062652   192 device_context.cc:362] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0704 22:53:11.067523   192 device_context.cc:372] device: 0, cuDNN Version: 7.6.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:143: DeprecationWarning:

# 7.提交结果
执行完预测程序后，会在跟目录生成结果csv文件，可用于比赛结果提交。